<a href="https://colab.research.google.com/github/kellytog/BI/blob/main/reto_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

0. cargar las librerias con las que vamos a trabajar

In [ ]:
# las dos librerias cientificas
import numpy as np
import pandas as pd

#las librerias del arbol
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix

#conectar al drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


1. cargamos la base de datos


In [ ]:
nxl='/content/drive/MyDrive/ANALITICA DE NEGOCIOS/2. CollegeAdmisions_Int_M.xlsx'
XDB=pd.read_excel(nxl,sheet_name=0)
XDB=XDB.dropna()

XDB['Gender'] = XDB['Gender'].astype('category').cat.codes
XDB['Admitted'] = XDB['Admitted'].astype('category').cat.codes

XD=XDB[['Edu_Parent1','Edu_Parent2','Gender','White','Asian','HSGPA','SAT/ACT','Admitted']]
#seleccionar las variables
XD=XDB[['Edu_Parent1','Edu_Parent2','Gender','White','Asian','HSGPA','SAT/ACT']]
yd=XDB[['Admitted']]

# Imprime las primeras filas para verificar
print(XD.head())

    Edu_Parent1  Edu_Parent2  Gender  White  Asian  HSGPA  SAT/ACT
6             6            6       0      1      0   4.26     1430
28            7            6       1      1      0   3.95     1340
46            3            6       1      1      0   4.33     1200
61            7            7       0      1      0   4.27     1530
65            6            7       0      0      0   4.04     1270


**Especificamos variables segun el libro**
1. Applicant: Identificador único del solicitante.
2. Edu_Parent1: Nivel educativo del primer padre (posiblemente en una escala numérica, donde valores más altos representan mayor educación).
3. Edu_Parent2: Nivel educativo del segundo padre.
4. Gender: Género del solicitante (M = Masculino, F = Femenino).
5. White: Indicador binario (0 o 1) de si el solicitante se identifica como blanco.
6. Asian: Indicador binario (0 o 1) de si el solicitante se identifica como asiático.
7. HSGPA: Promedio de calificaciones en la escuela secundaria (escala de 0 a 4 o superior).
8. SAT/ACT: Puntaje combinado en exámenes estandarizados (SAT o ACT).
9. College: Área de estudio de interés (Ej: "Business & Economics", "Math & Science", etc.).
10. Admitted: Indicador de si el solicitante fue admitido (Yes/No).
11. Enrolled: Indicador de si el solicitante se inscribió tras ser admitido (Yes/No).
12. College_GPA: Promedio de calificaciones en la universidad (si el estudiante se inscribió).

**descripcion de los datos**
De acuerdo con la base de datos de solicitantes de college admission, esta posee un total de 1528 datos totales divididos en 225 negados y 1303 admitidos, cada uno de los solicitantes de admision estara descrito en funcion de 7 variables que fueron especificadas en la parte superior que son 'Edu_Parent1','Edu_Parent2','Gender','White','Asian','HSGPA','SAT/ACT' que van a se consideradas nuestras variables de entrada y 'Admitted' como nuestra variable de salida, es decir, aquella variable que buscamos sea pronosticada por el modelo. en esta ocasion, dejamos por fuera las variables "Enrolled" y "College_GPA" debido a su poca variación en relación con la decisión de admisión. Ademas, cuando nos acercamos a visualizar nuestro arbol de decision, pordemos encontrar el indicador gini en 0 en 7 ocasiones, esto es muy positivo ya que quiere decir que en 7 tipos de casos posibles nuestro modelo es 100% efectivo a la hora de tomar una decision

2. HACEMOS EL TREE CLASSIFIER

In [ ]:
mar=DecisionTreeClassifier(criterion='gini',max_depth=4)
mar.fit(XD,yd)

#se procede con la grafica
from pydotplus import graph_from_dot_data #me crea la grafica
from sklearn.tree import export_graphviz#me exporta a PNG, JPEG,TIFF

ve=['Edu_Parent1', 'Edu_Parent2', 'Gender', 'White', 'Asian', 'HSGPA', 'SAT/ACT'] #nombres para el arbol
dot_data=export_graphviz(mar,feature_names=ve) #le dije que exporte el arbol y que le coloque los nombres para el grafico
graph=graph_from_dot_data(dot_data)
graph.write_png('2.ArbolDecision_reto.png')

True

Tomamos 4 niveles de análisis para el árbol de decisión, con el fin de identificar quienes han sido admitidos, esto se hace con max_depth

3. aqui vamos a pronosticar si se aprueba o se niega a los solicitantes

In [ ]:
XDB0=pd.read_excel(nxl,sheet_name=1) #leemos la hoja del excel
XDB0.head(100)

XDB0['Gender'] = XDB0['Gender'].astype('category').cat.codes

#se crea la base de datos a pronosticar
XD0=XDB0[['Edu_Parent1','Edu_Parent2','Gender','White','Asian','HSGPA','SAT/ACT']]

#LE CALCULAMOS LA PREAPROBACION A LOS SOLICITANTES
y_pred=mar.predict(XD0)

#se crea el informe de excel
df=pd.DataFrame(np.column_stack((XD0,y_pred))) #armamos tabla filas para los que no tienen pre, las variables con XD0 y y_pred para el pronostico
df.columns=['Edu_Parent1','Edu_Parent2','Gender','White','Asian','HSGPA','SAT/ACT','Admitted']
df.to_excel("Evaluacion aprobados.xlsx")

4. Aqui evaluamos el modelo

In [ ]:
from re import S
ydp=mar.predict(XD) #PRONOSTICO SOBRE LA BASE DE DATOS ORIGINAL
cm=confusion_matrix(yd,ydp)
print("La matriz de  confusion es:\n",cm)

VN=cm[0,0];FP=cm[0,1];FN=cm[1,0];VP=cm[1,1]
Exactitud=(VP+VN)/(VP+VN+FP+FN)
Sensibilidad=VP/(VP+FN)
Especificidad=VN/(VN+FP)
Precision=VP/(VP+FP)
TasaError=(FP+FN)/(VP+VN+FP+FN)
predneg=VN/(VN+FN)
print("Tasa de error:",TasaError)
print("Tasa de prediccion negativa:",predneg)
print("Exactitud:",Exactitud)
print("Sensibilidad:",Sensibilidad)
print("Especificidad:",Especificidad)
print("Precision:",Precision)



La matriz de  confusion es:
 [[  20  205]
 [   2 1301]]
Tasa de error: 0.1354712041884817
Tasa de prediccion negativa: 0.9090909090909091
Exactitud: 0.8645287958115183
Sensibilidad: 0.9984650805832693
Especificidad: 0.08888888888888889
Precision: 0.8638778220451527


**ANALISIS DE DATOS**
la matriz de confusion muestra una predominancia por los valores verdaderos positivos que son 1301 a cmparacion de loa demas resultados que se pronostica un valor mucho menos, esto quiere decir que el modelo predice correctamente a la mayoría de los estudiantes admitidos, el siguiente valor son los falsos positivos con un valor de 205, El número de falsos negativos es bajo (2), lo cual es bueno, ya que indica que el modelo no está perdiendo a muchos estudiantes que realmente deberían ser admitidos. Lo anterior hizo que el modelo alcanzara valores por encima del 85% en promedio para los indices exactitud,sensibilidad, precision, lo que muestra claramente el buen funcionamiento del modelo frente al pronostico de la admision, La especificidad es muy baja (8.89%), lo que significa que el modelo tiene dificultades para identificar correctamente a los estudiantes que no deberían ser admitidos.
El modelo muestra una sensibilidad muy cercana al 100% (99.85%), lo que indica que el modelo es excelente para identificar a los estudiantes que realmente deberían ser admitidos. una exactitud de 86.45%, lo que significa que predice correctamente la admisión en general en ese porcentaje de casos y con un valor muy similar a la exactitud tenemos La precisión, es del 86.39%, lo que indica que, de todos los estudiantes que el modelo predice que serán admitidos, el 86.39% realmente lo son.